# Running 2_PT_Senseflux_intersectionpoint for the whole year each two month

In [37]:
# define path to FVCOM data files
import xarray as xr
import os
FVCOM_DIR = '/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2024_corrected/'

In [8]:
# check if the data exist in the directory
all_files = sorted([f for f in os.listdir(FVCOM_DIR) if f.endswith(".nc")])

print("Earliest file:", all_files[0])

Earliest file: nos.lmhofs.fields.f000.20231119.t18z.nc


In [ ]:

FVCOM_DIR = "/home/abolmaal/modelling/FVCOM/Huron/output/2024"
#filename = "particleload_updated_FVCOM_Huron_2323_JanFeb_1.nc"



In [39]:

import cftime
from netCDF4 import num2date
filename = 'lmhofs.fields.n006.20241231.t00z.nc'

ds = xr.open_dataset(FVCOM_DIR + filename, decode_times=False)
time_var = ds.variables['time']

# Extract the values and metadata properly
time_vals = time_var[:]
units = time_var.attrs.get('units', None)
calendar = time_var.attrs.get('calendar', 'standard')

if units is None:
    raise ValueError("The 'units' attribute is missing from the time variable.")

# Convert to datetime
converted_time = num2date(time_vals, units=units, calendar=calendar)

print("Start time:", converted_time[0])
print("End time:", converted_time[-1])


Start time: 2024-12-31 00:00:00
End time: 2024-12-31 00:00:00


In [ ]:
for fname in sorted(os.listdir(FVCOM_DIR)):
    if fname.endswith('.nc'):
        ds = xr.open_dataset(os.path.join(FVCOM_DIR, fname), decode_times=False)
        print(f"{fname}: {ds['time'].values[0]} to {ds['time'].values[-1]}")
        ds.close()

## running the model

In [40]:
import os
import glob
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import datetime
import configparser
import subprocess
# FVCOM-specific visualization and utility tools
from pylag.processing.plot import FVCOMPlotter, create_figure, colourmap
from pylag.processing.utils import get_grid_bands
from pylag.grid_metrics import create_fvcom_grid_metrics_file

# Regridding, viewing, and garbage collection utilities
from pylag.regrid import regridder
from pylag.processing.ncview import Viewer


# Function to generate start and end datetimes for each two-month period
def generate_date_ranges(year):
    # Adjust to create correct date ranges from August to December
    #months = [(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8, 9),(9, 10),(10, 11),(11, 12),]
    months = [(11,12)]
    
    date_ranges = []
    for start_month, end_month in months:
        start_datetime = datetime.datetime(year, start_month, 1)
        
        # Special case for September: Set end date to 29th
        if start_month == 1:
            start_datetime = datetime.datetime(year, start_month, 1, 18, 00, 00)
        if start_month == 8:
            start_datetime = datetime.datetime(year, start_month, 1, 6, 00, 00)
        if start_month == 11:
            start_datetime = datetime.datetime(year, start_month, 1, 00, 00, 00)
        if end_month == 2:
            end_datetime = datetime.datetime(year, end_month, 28, 12, 00, 00)
        if end_month == 10:
            end_datetime = datetime.datetime(year, end_month, 30, 00, 00, 00)
        if end_month == 8:
            end_datetime = datetime.datetime(year, end_month, 31, 6, 00, 00)
        if end_month == 11:
            end_datetime = datetime.datetime(year, end_month, 30, 18, 00, 00)
        elif end_month == 12:
            end_datetime = datetime.datetime(year, end_month, 30, 18, 00, 00)
        else:
            end_datetime = datetime.datetime(year, end_month + 1, 1) - datetime.timedelta(seconds=1)
        
        date_ranges.append((start_datetime, end_datetime))
    
    return date_ranges

# Function to update the datetime and output filename in the config file
def update_datetime_in_config(config_file_name, start_str, end_str, out_dir):
    start_date = datetime.datetime.strptime(start_str, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.datetime.strptime(end_str, '%Y-%m-%d %H:%M:%S')
    
    start_year = start_date.year
    start_month = start_date.month
    end_year = end_date.year
    end_month = end_date.month
    
    month_range = f"{start_date.strftime('%b')}{end_date.strftime('%b')}"
    output_filename = f"FVCOM_Huron_{start_year % 100}{end_year % 100}_{month_range}"

    # Read the config file and update the necessary values
    with open(config_file_name, 'r') as file:
        lines = file.readlines()

    with open(config_file_name, 'w') as file:
        for line in lines:
            if line.strip().startswith("start_datetime"):
                file.write(f"start_datetime = {start_str}\n")
            elif line.strip().startswith("end_datetime"):
                file.write(f"end_datetime = {end_str}\n")
            elif line.strip().startswith("output_file"):
                # Ensure the output file name is updated correctly
                file.write(f"output_file = %(out_dir)s/{output_filename}\n")
            else:
                file.write(line)

    # Now, update the pylag.cfg with the same changes
    pylag_cfg_path = os.path.join(out_dir, 'pylag.cfg')
    with open(pylag_cfg_path, 'w') as config_file:
        for line in lines:
            if line.strip().startswith("start_datetime"):
                config_file.write(f"start_datetime = {start_str}\n")
            elif line.strip().startswith("end_datetime"):
                config_file.write(f"end_datetime = {end_str}\n")
            elif line.strip().startswith("output_file"):
                config_file.write(f"output_file = %(out_dir)s/{output_filename}\n")
            else:
                config_file.write(line)

# Main script
def main():
    # Define directories and file paths
    config_file_path = '/home/abolmaal/modelling/FVCOM/Huron/config_files'
    config_file_name = os.path.join(config_file_path, 'Huron_Senseflux_Seasonal.cfg')
    MODELLING_DIR = '/home/abolmaal/modelling/FVCOM/Huron'
    out_dir = os.path.join(MODELLING_DIR, 'output')
    FVCOM_DIR = '/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2024_corrected'
    input_dir = os.path.join(MODELLING_DIR, 'input')
    grid_metrics_file_name = f'{input_dir}/gridfile/grid_metrics_huron_senseflux_Seasonal.nc'
    pylag_cfg_path = os.path.join(out_dir, 'pylag.cfg')

    # Create necessary directories
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)

    # Initialize the configuration parser
    cf = configparser.ConfigParser()
    cf.read(config_file_name)

    # Set configuration parameters
    cf.set('OCEAN_DATA', 'data_dir', FVCOM_DIR)
    cf.set('OCEAN_DATA', 'grid_metrics_file', grid_metrics_file_name)
    cf.set('GENERAL', 'out_dir', out_dir)

    # Save the updated configuration to pylag.cfg
    with open(pylag_cfg_path, 'w') as config_file:
        cf.write(config_file)

    print(f"Updated configuration and saved to {pylag_cfg_path}")

    # Generate date ranges for the year
    year = 2024
    date_ranges = generate_date_ranges(year)

    # Loop through each date range and run the model
    for start_datetime, end_datetime in date_ranges:
        start_str = start_datetime.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end_datetime.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Running for: {start_str} to {end_str}")
        
        # Update the datetime in the config file and pylag.cfg file
        update_datetime_in_config(config_file_name, start_str, end_str, out_dir)
        print(f"Updated start_datetime, end_datetime, and output_file in {config_file_name} and pylag.cfg.")
        
        # Print the output filename to confirm it's unique for each run
        print(f"Output file for this run: {start_str[:7]}_run.nc")  # Example: FVCOM_Huron_2323_JanFeb_run.nc
        
        # Change to the modeling directory
        os.chdir(MODELLING_DIR)

        # Run the model using the updated pylag.cfg file with a new output path
        run_command = f"python -m pylag.main -c {pylag_cfg_path}"
        print(f"Running model with command: {run_command}")

        # Execute the model with subprocess and capture output
        try:
            result = subprocess.run(run_command, shell=True, check=True, capture_output=True, text=True)
            print("Model run completed.")
            print(f"stdout: {result.stdout}")
            print(f"stderr: {result.stderr}")
        except subprocess.CalledProcessError as e:
            print(f"Error occurred while running the model:")
            print(f"stdout: {e.stdout}")
            print(f"stderr: {e.stderr}")
            print(f"Return code: {e.returncode}")

if __name__ == "__main__":
    main()

Updated configuration and saved to /home/abolmaal/modelling/FVCOM/Huron/output/pylag.cfg
Running for: 2024-11-01 00:00:00 to 2024-12-30 18:00:00
Updated start_datetime, end_datetime, and output_file in /home/abolmaal/modelling/FVCOM/Huron/config_files/Huron_Senseflux_Seasonal.cfg and pylag.cfg.
Output file for this run: 2024-11_run.nc
Running model with command: python -m pylag.main -c /home/abolmaal/modelling/FVCOM/Huron/output/pylag.cfg
Model run completed.
stdout: 
Starting ensemble member 1 ...
Progress:

stderr:   0% |                                           |
  1% |                                           |
  2% |                                           |
  3% |#                                          |
  4% |#                                          |
  5% |##                                         |
  6% |##                                         |
  7% |###                                        |
  8% |###                                        |
  9% |###         